In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1388817,2021-06-20,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1388818,2021-06-21,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1388819,2021-06-22,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1388820,2021-06-23,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
36743,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
36745,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
36747,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
36749,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
36751,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
851236,2021-06-20,Arkansas,Arkansas,5001,2176,38.00,5000,Arkansas,AR,Arkansas,State,3017804
851238,2021-06-21,Arkansas,Arkansas,5001,2176,38.00,5000,Arkansas,AR,Arkansas,State,3017804
851240,2021-06-22,Arkansas,Arkansas,5001,2186,38.00,5000,Arkansas,AR,Arkansas,State,3017804
851242,2021-06-23,Arkansas,Arkansas,5001,2190,38.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1388817,2021-06-20,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1388818,2021-06-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1388819,2021-06-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1388820,2021-06-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-06-24') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
520,2021-06-24,Snohomish,Washington,39984,599.00,53061,WA,County,822083,4863.74,72.86
1038,2021-06-24,Cook,Illinois,556629,11030.00,17031,IL,County,5150233,10807.84,214.17
1555,2021-06-24,Orange,California,273170,5116.00,6059,CA,County,3175692,8601.90,161.10
2071,2021-06-24,Maricopa,Arizona,558068,10243.00,4013,AZ,County,4485414,12441.84,228.36
2587,2021-06-24,Los Angeles,California,1249083,24472.00,6037,CA,County,10039107,12442.17,243.77
...,...,...,...,...,...,...,...,...,...,...,...
1387925,2021-06-24,Wheeler,Oregon,35,1.00,41069,OR,County,1332,2627.63,75.08
1388180,2021-06-24,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1388404,2021-06-24,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1388624,2021-06-24,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
520,2021-06-24,Snohomish,Washington,39984,599.00,53061,WA,County,822083,4863.74,72.86,72.86,4863.74
1038,2021-06-24,Cook,Illinois,556629,11030.00,17031,IL,County,5150233,10807.84,214.17,214.17,10807.84
1555,2021-06-24,Orange,California,273170,5116.00,6059,CA,County,3175692,8601.90,161.10,161.10,8601.90
2071,2021-06-24,Maricopa,Arizona,558068,10243.00,4013,AZ,County,4485414,12441.84,228.36,228.36,12441.84
2587,2021-06-24,Los Angeles,California,1249083,24472.00,6037,CA,County,10039107,12442.17,243.77,243.77,12442.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387925,2021-06-24,Wheeler,Oregon,35,1.00,41069,OR,County,1332,2627.63,75.08,75.08,2627.63
1388180,2021-06-24,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1388404,2021-06-24,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1388624,2021-06-24,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00,0.00,1183.43


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
890199,2021-06-24,Hale,Alabama,2270,78.00,1065,AL,County,14651,15493.82,532.39,532.39,15493.82,1
969219,2021-06-24,Clarke,Alabama,3533,61.00,1025,AL,County,23622,14956.40,258.23,258.23,14956.40,2
696081,2021-06-24,Lowndes,Alabama,1421,54.00,1085,AL,County,9726,14610.32,555.21,555.21,14610.32,3
617026,2021-06-24,Etowah,Alabama,14192,364.00,1055,AL,County,102268,13877.26,355.93,355.93,13877.26,4
517566,2021-06-24,Franklin,Alabama,4320,82.00,1059,AL,County,31362,13774.63,261.46,261.46,13774.63,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313979,2021-06-24,Platte,Wyoming,679,12.00,56031,WY,County,8393,8090.08,142.98,142.98,8090.08,19
849431,2021-06-24,Converse,Wyoming,1034,18.00,56009,WY,County,13822,7480.83,130.23,130.23,7480.83,20
1056776,2021-06-24,Lincoln,Wyoming,1422,14.00,56023,WY,County,19830,7170.95,70.60,70.60,7170.95,21
1176742,2021-06-24,Niobrara,Wyoming,163,2.00,56027,WY,County,2356,6918.51,84.89,84.89,6918.51,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
696081,2021-06-24,Lowndes,Alabama,1421,54.00,1085,AL,County,9726,14610.32,555.21,555.21,14610.32,3,1
890199,2021-06-24,Hale,Alabama,2270,78.00,1065,AL,County,14651,15493.82,532.39,532.39,15493.82,1,2
615198,2021-06-24,Clay,Alabama,1601,59.00,1027,AL,County,13235,12096.71,445.79,445.79,12096.71,20,3
295545,2021-06-24,Walker,Alabama,7351,281.00,1127,AL,County,63521,11572.55,442.37,442.37,11572.55,28,4
616569,2021-06-24,Dallas,Alabama,3609,158.00,1047,AL,County,37196,9702.66,424.78,424.78,9702.66,55,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056776,2021-06-24,Lincoln,Wyoming,1422,14.00,56023,WY,County,19830,7170.95,70.60,70.60,7170.95,21,19
748977,2021-06-24,Hot Springs,Wyoming,372,3.00,56017,WY,County,4413,8429.64,67.98,67.98,8429.64,17,20
1030344,2021-06-24,Uinta,Wyoming,2305,13.00,56041,WY,County,20226,11396.22,64.27,64.27,11396.22,5,21
294156,2021-06-24,Teton,Wyoming,3798,11.00,56039,WY,County,23464,16186.50,46.88,46.88,16186.50,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,72.86,4863.74,21,25
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,72.86,4863.74,21,25
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,72.86,4863.74,21,25
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,72.86,4863.74,21,25
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,72.86,4863.74,21,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1385576,2021-06-20,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1385577,2021-06-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1385578,2021-06-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1385579,2021-06-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
886507,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15493.82,2,1,1.00
886508,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15493.82,2,1,0.00
886509,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15493.82,2,1,0.00
886510,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15493.82,2,1,0.00
886511,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,532.39,15493.82,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074540,2021-06-20,Crook,Wyoming,457,12.00,56011,WY,County,7584,6025.84,158.23,158.23,6039.03,7,23,0.00
1074541,2021-06-21,Crook,Wyoming,458,12.00,56011,WY,County,7584,6039.03,158.23,158.23,6039.03,7,23,1.00
1074542,2021-06-22,Crook,Wyoming,458,12.00,56011,WY,County,7584,6039.03,158.23,158.23,6039.03,7,23,0.00
1074543,2021-06-23,Crook,Wyoming,458,12.00,56011,WY,County,7584,6039.03,158.23,158.23,6039.03,7,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
693293,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14610.32,1,3,1.00,0.00
693294,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14610.32,1,3,0.00,0.00
693295,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14610.32,1,3,0.00,0.00
693296,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14610.32,1,3,0.00,0.00
693297,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14610.32,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691008,2021-06-20,Albany,Wyoming,4583,15.00,56001,WY,County,38880,11787.55,38.58,43.72,11849.28,23,4,0.00,0.00
691009,2021-06-21,Albany,Wyoming,4591,15.00,56001,WY,County,38880,11808.13,38.58,43.72,11849.28,23,4,8.00,0.00
691010,2021-06-22,Albany,Wyoming,4595,15.00,56001,WY,County,38880,11818.42,38.58,43.72,11849.28,23,4,4.00,0.00
691011,2021-06-23,Albany,Wyoming,4602,17.00,56001,WY,County,38880,11836.42,43.72,43.72,11849.28,23,4,7.00,2.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-06-24') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
352925,2021-06-24,Imperial,California,28908,741.00,6025,CA,County,181215,15952.32,408.91,408.91,15952.32,1,2,28.00,0.00
2587,2021-06-24,Los Angeles,California,1249083,24472.00,6037,CA,County,10039107,12442.17,243.77,243.77,12442.17,2,5,287.00,11.00
175517,2021-06-24,San Bernardino,California,299425,4886.00,6071,CA,County,2180085,13734.56,224.12,224.12,13734.56,3,4,53.00,13.00
618806,2021-06-24,Inyo,California,1434,38.00,6027,CA,County,18039,7949.44,210.65,210.65,7949.44,4,25,0.00,0.00
78381,2021-06-24,Stanislaus,California,63586,1071.00,6099,CA,County,550660,11547.23,194.49,194.49,11547.23,5,9,28.00,1.00
62335,2021-06-24,San Joaquin,California,74808,1440.00,6077,CA,County,762148,9815.42,188.94,188.94,9815.42,6,17,0.00,0.00
36267,2021-06-24,Riverside,California,301799,4627.00,6065,CA,County,2470546,12215.88,187.29,187.29,12215.88,7,7,0.00,0.00
97218,2021-06-24,Tulare,California,49823,850.00,6107,CA,County,466195,10687.16,182.33,182.33,10687.16,8,10,14.00,0.00
560657,2021-06-24,Merced,California,32325,481.00,6047,CA,County,277680,11641.10,173.22,173.22,11641.10,9,8,8.00,0.00
35317,2021-06-24,Fresno,California,103020,1724.00,6019,CA,County,999101,10311.27,172.56,172.56,10311.27,10,14,24.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1313530,2021-06-24,Lassen,California,5799,24.00,6035,CA,County,30573,18967.72,78.50,78.50,18967.72,36,1,0.00,0.00
352925,2021-06-24,Imperial,California,28908,741.00,6025,CA,County,181215,15952.32,408.91,408.91,15952.32,1,2,28.00,0.00
750740,2021-06-24,Kings,California,23214,247.00,6031,CA,County,152940,15178.50,161.50,161.50,15178.50,11,3,5.00,0.00
175517,2021-06-24,San Bernardino,California,299425,4886.00,6071,CA,County,2180085,13734.56,224.12,224.12,13734.56,3,4,53.00,13.00
2587,2021-06-24,Los Angeles,California,1249083,24472.00,6037,CA,County,10039107,12442.17,243.77,243.77,12442.17,2,5,287.00,11.00
220761,2021-06-24,Kern,California,110847,1405.00,6029,CA,County,900202,12313.57,156.08,156.08,12313.57,13,6,0.00,0.00
36267,2021-06-24,Riverside,California,301799,4627.00,6065,CA,County,2470546,12215.88,187.29,187.29,12215.88,7,7,0.00,0.00
560657,2021-06-24,Merced,California,32325,481.00,6047,CA,County,277680,11641.10,173.22,173.22,11641.10,9,8,8.00,0.00
78381,2021-06-24,Stanislaus,California,63586,1071.00,6099,CA,County,550660,11547.23,194.49,194.49,11547.23,5,9,28.00,1.00
97218,2021-06-24,Tulare,California,49823,850.00,6107,CA,County,466195,10687.16,182.33,182.33,10687.16,8,10,14.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
352925,2021-06-24,Imperial,California,28908,741.00,6025,CA,County,181215,15952.32,408.91,408.91,15952.32,1,2,28.00,0.00
2587,2021-06-24,Los Angeles,California,1249083,24472.00,6037,CA,County,10039107,12442.17,243.77,243.77,12442.17,2,5,287.00,11.00
175517,2021-06-24,San Bernardino,California,299425,4886.00,6071,CA,County,2180085,13734.56,224.12,224.12,13734.56,3,4,53.00,13.00
618806,2021-06-24,Inyo,California,1434,38.00,6027,CA,County,18039,7949.44,210.65,210.65,7949.44,4,25,0.00,0.00
78381,2021-06-24,Stanislaus,California,63586,1071.00,6099,CA,County,550660,11547.23,194.49,194.49,11547.23,5,9,28.00,1.00
62335,2021-06-24,San Joaquin,California,74808,1440.00,6077,CA,County,762148,9815.42,188.94,188.94,9815.42,6,17,0.00,0.00
36267,2021-06-24,Riverside,California,301799,4627.00,6065,CA,County,2470546,12215.88,187.29,187.29,12215.88,7,7,0.00,0.00
97218,2021-06-24,Tulare,California,49823,850.00,6107,CA,County,466195,10687.16,182.33,182.33,10687.16,8,10,14.00,0.00
560657,2021-06-24,Merced,California,32325,481.00,6047,CA,County,277680,11641.10,173.22,173.22,11641.10,9,8,8.00,0.00
35317,2021-06-24,Fresno,California,103020,1724.00,6019,CA,County,999101,10311.27,172.56,172.56,10311.27,10,14,24.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,408.91,15952.32,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,408.91,15952.32,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,408.91,15952.32,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,408.91,15952.32,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,408.91,15952.32,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4632,06/20/21,Lassen,5784,24.00,18918.65,78.50,78.50,18967.72,36,1,0.00,0.00
4633,06/21/21,Lassen,5784,24.00,18918.65,78.50,78.50,18967.72,36,1,0.00,0.00
4634,06/22/21,Lassen,5799,24.00,18967.72,78.50,78.50,18967.72,36,1,15.00,0.00
4635,06/23/21,Lassen,5799,24.00,18967.72,78.50,78.50,18967.72,36,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,408.91,15952.32,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,408.91,15952.32,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,408.91,15952.32,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,408.91,15952.32,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,408.91,15952.32,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4718,06/20/21,Fresno,102922,1724.00,10301.46,172.56,172.56,10311.27,10,14,28.00,2.00
4719,06/21/21,Fresno,102947,1724.00,10303.96,172.56,172.56,10311.27,10,14,25.00,0.00
4720,06/22/21,Fresno,102971,1724.00,10306.37,172.56,172.56,10311.27,10,14,24.00,0.00
4721,06/23/21,Fresno,102996,1724.00,10308.87,172.56,172.56,10311.27,10,14,25.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)